<a href="https://colab.research.google.com/github/stuart-lane/MachineLearning/blob/main/LogitSimulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import random

np.random.seed(123456)

DATA GENERATING PROCESS

In [4]:
n = 1000

x = np.random.randn(n)
u = np.random.randn(n)
beta0 = 0
beta1 = 1

p = 1 / (1 + np.exp(-(beta0 + x*beta1 + u)))
y = np.random.binomial(n = 1, p = p, size = n)

In [5]:
from sklearn.model_selection import train_test_split

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 123)

LOGISTIC REGRESSION WITH SKLEARN

In [7]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=1000)  # You can change the solver and max_iter as needed
model.fit(x_train, y_train)

In [9]:
intercept = model.intercept_[0]
coefficient = model.coef_[0][1]

In [10]:
print(f"Intercept: {intercept}")
print(f"Coefficient for 'Regressor': {coefficient}")

Intercept: 0.13604864738861402
Coefficient for 'Regressor': 1.0227435494366917


STANDARD LOGISTIC REGRESSION

In [11]:
# Define the logistic function
def logistic(x, beta):
    x_beta = np.dot(x, beta)
    x_beta = np.clip(x_beta, -500, 500)
    return 1 / (1 + np.exp(-x_beta))

# Define the log-likelihood function
def log_likelihood(beta, x, y):
    p = logistic(x, beta)
    epsilon = 1e-10
    p = np.clip(p, epsilon, 1 - epsilon)
    ll = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
    return -ll

beta_initial = np.array([0, 0])


from scipy.optimize import minimize
result = minimize(log_likelihood, beta_initial, args=(x_train, y_train), method = 'CG')

print(result)

estimated_beta = [f'{param:.7f}' for param in result.x]
formatted_parameters = ', '.join(estimated_beta)

print(f"Estimated Parameters: [{formatted_parameters}]")
print("Negative Log-Likelihood:", result.fun)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 173158.77328570024
       x: [ 6.402e-02 -2.021e-08]
     nit: 5
     jac: [ 0.000e+00  0.000e+00]
    nfev: 36
    njev: 12
Estimated Parameters: [0.0640219, -0.0000000]
Negative Log-Likelihood: 173158.77328570024


In [18]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for i in range(num_iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= learning_rate * gradient

    return theta

learning_rate = 0.1
num_iterations = 1000

theta = logistic_regression(x_train, y_train, learning_rate, num_iterations)

print("Estimated Parameters (Weights):", theta)

Estimated Parameters (Weights): [[0.13685842]
 [1.03705832]]


AUTOGRAD

In [19]:
import torch
from torch.optim import LBFGS
from torch.optim import Adam

In [20]:
train_df_keys = {'constant': x_train[:,0].flatten(), 'X1': x_train[:,1].flatten(), 'y': y_train.flatten()}

train_df = pd.DataFrame(train_df_keys)

In [21]:
x = torch.cat((torch.tensor(train_df[["constant", "X1"]].values, dtype=torch.float64),), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["y"].values, dtype = torch.float64)
y.requires_grad = True

beta_initial = np.array([0, 0])
beta = torch.zeros(2, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

Running on: cpu


In [22]:
# Define the loss function
def ll(beta):
    xbetahat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbetahat)
    loss = - torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

# Set up the Adam optimizer
learning_rate = 0.01
optimizer = LBFGS([beta], lr = learning_rate)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = ll(beta)
    value.backward()
    return value

In [31]:
for i in range(1, 10):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  print(f"{i}: {logl_value}")
  print(f"{i}: {estimated_weights}")

estimated_weights = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights)

1: 372.34156723584204
1: [[-0.74413014]
 [ 0.14729873]]
2: 372.34156723584204
2: [[-0.74413014]
 [ 0.14729873]]
3: 372.34156723584204
3: [[-0.74413014]
 [ 0.14729873]]
4: 372.34156723584204
4: [[-0.74413014]
 [ 0.14729873]]
5: 372.34156723584204
5: [[-0.74413014]
 [ 0.14729873]]
6: 372.34156723584204
6: [[-0.74413014]
 [ 0.14729873]]
7: 372.34156723584204
7: [[-0.74413014]
 [ 0.14729873]]
8: 372.34156723584204
8: [[-0.74413014]
 [ 0.14729873]]
9: 372.34156723584204
9: [[-0.74413014]
 [ 0.14729873]]
Estimated Parameters (Weights): [[-0.74413014]
 [ 0.14729873]]


In [24]:
# Define the initial values
debugstart = torch.tensor([1, 2, 3], dtype = torch.float64, requires_grad = True)
x0 = torch.tensor([1, 1/2, 1/3], dtype = torch.float64, requires_grad = True)
y1 = torch.tensor([1], dtype = torch.float64, requires_grad = True)

# Calculate xbhat
xbhat = torch.mm(x0.unsqueeze(0), debugstart.unsqueeze(1))
xbhat.retain_grad()

# Calculate yhat
yhat = torch.sigmoid(xbhat)
yhat.retain_grad()

# Calculate the loss
loss = -torch.sum(y1 * torch.log(yhat) + (1 - y1) * torch.log(1 - yhat))
loss.backward()

# Print the tensors and their gradients
print("y:", y1)
print("x:", x0)
print("xbhat:", xbhat)
print("Gradients of xbhat (dloss/dp):", xbhat.grad)
print("yhat:", yhat)
print("Gradients of yhat (dloss/dyhat):", yhat.grad)
print("Loss:", loss.item())

y: tensor([1.], dtype=torch.float64, requires_grad=True)
x: tensor([1.0000, 0.5000, 0.3333], dtype=torch.float64, requires_grad=True)
xbhat: tensor([[3.]], dtype=torch.float64, grad_fn=<MmBackward0>)
Gradients of xbhat (dloss/dp): tensor([[-0.0474]], dtype=torch.float64)
yhat: tensor([[0.9526]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
Gradients of yhat (dloss/dyhat): tensor([[-1.0498]], dtype=torch.float64)
Loss: 0.04858735157374191


NEURAL NETWORK LOGIT

In [25]:
import torch.nn as nn
import torch.nn.functional as F

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple logistic regression model
class LogitModel(nn.Module):
    def __init__(self, input_dim):
        super(LogitModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Single output unit

    def forward(self, x):
        x = x.to(self.linear.weight.dtype)  # Cast input to the same data type as the model's parameters
        x = self.linear(x)
        x = torch.sigmoid(x)  # Apply sigmoid activation for binary classification
        return x

# Assuming you have 'train' DataFrame with 'balance' and 'income' columns
x = torch.tensor(x_train, dtype = torch.float64)
x.requires_grad = True

y = torch.tensor(y_train, dtype = torch.float64)
y.requires_grad = True
y = y.reshape(-1, 1)

beta = torch.randn(x.shape[1], 1, dtype = torch.float64)
beta.requires_grad = True

# Define model input dimension based on the number of features in your data
input_dim = x.shape[1]

# Create an instance of the LogitModel
model = LogitModel(input_dim)

# Define a binary cross-entropy loss function
loss_function = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.01)

# Set the number of training epochs
epochs = 5000

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    outputs = model(x)

    outputs = outputs.float()
    y = y.float()

    # Calculate the loss
    loss = loss_function(outputs, y)

    # Backpropagation
    loss.backward()

    # Update the model's parameters
    optimizer.step()

    # Print training progress
    if epoch % (epochs / 10) == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

# View the learned parameters
print("Learned parameters:")
print(f"{model.linear.weight}") # these are the parameters of interest
print(f"{model.linear.bias}")

Epoch [0/5000], Loss: 0.6974
Epoch [500/5000], Loss: 0.5904
Epoch [1000/5000], Loss: 0.5904
Epoch [1500/5000], Loss: 0.5904
Epoch [2000/5000], Loss: 0.5904
Epoch [2500/5000], Loss: 0.5904
Epoch [3000/5000], Loss: 0.5904
Epoch [3500/5000], Loss: 0.5904
Epoch [4000/5000], Loss: 0.5904
Epoch [4500/5000], Loss: 0.5904
Learned parameters:
Parameter containing:
tensor([[-0.2359,  1.0371]], requires_grad=True)
Parameter containing:
tensor([0.3728], requires_grad=True)
